In [ ]:
#This function calculates the X and Y centroid of a given image, if the moment 
#or the centroid of the image is 0, the function returns -1
def centroid(img):
    #Make the image grayscale
    R, G, B = img[:,:,0], img[:,:,1], img[:,:,2]
    imgGray = 0.2989 * R + 0.5870 * G + 0.1140 * B

    #Convert to binary
    ret,thresh = cv2.threshold(imgGray,127,255,0)

    #Calculate moment
    M = cv2.moments(thresh)

    #Calculate centroid position
    if M["m00"] != 0:
      Xc = int(M["m10"] / M["m00"])
      Yc = int(M["m01"] / M["m00"])

      if Xc == 0 or Yc == 0:
        Xc = -1
        Yc = -1
    else:
      #TODO: Skip frame
      Xc = -1
      Yc = -1

    return Xc, Yc

In [ ]:
#Find key frame in a sequence of frames and return the position relative to the sequence
def findKeyFrame(sequence):
  X_seq = 0
  Y_seq = 0

  for frame in sequence:
    Xc, Yc = centroid(frame)
    X_seq += Xc
    Y_seq += Yc

  #Average centroid of the sequence
  X_seq = X_seq / len(sequence)
  Y_seq = Y_seq / len(sequence)

  #Calculate the centroid with the smallest distance from the average
  minDistance = math.inf
  position = 0
  cont = 0
  for frame in sequence:
    Xc, Yc = centroid(frame)
    tempDistance = (Xc - X_seq)**2 + (Yc - Y_seq)**2
    if tempDistance < minDistance:
      minDistance = tempDistance
      position = cont
    cont += 1

  return (len(sequence) - position)

In [ ]:
def createClip(keyFramePosition, videoFrames, keyFrames, clip_size):
  startPoint = keyFramePosition - (clip_size / 2)

  if startPoint < 0:
    startPoint = 0

  endPoint = startPoint + clip_size

  if endPoint > len(videoFrames):
    startPoint = keyFramePosition - clip_size
    endPoint = startPoint + clip_size

  print("StartPoint: ", startPoint, "  EndPoint", endPoint - 1)
  while startPoint < endPoint:
    keyFrames.append(videoFrames[int(startPoint)])
    startPoint += 1

  return keyFrames

In [ ]:
import os
import numpy as np
import cv2
import math
import matplotlib.pyplot as plt

def keyFramesExtraction(path,out_path, clip_size = 8):
  m = 14     #Minimum number of frames with visual similarities, you can tweek this value, the bigger the number
             #the more keyframes you are gonna find in the video

  e = 0.015  #Threshold for similarity (1.5%) this value need to be between 1% and 3%
  cont = 0
  i = 0

  videoCounter = 0

  for j in os.listdir(path):
    videoCounter += 1
    print("Elaborating video number ", videoCounter, " --------------------------------------------------------------------------------------------------------------")

    vid_path = os.path.join(path,j)
    vidcap = cv2.VideoCapture(vid_path)

    success,img = vidcap.read()

    if success == False:
      break

    sequence = []
    keyFrames = []
    videoFrames = []
    keyFramesFrameCounter = []

    similarCount = 0
    frameCounter = 1
    framesToSkip = 0
    sequence.append(img)
    videoFrames.append(img)
    keyFramesFrameCounter.append(frameCounter)

    Xc_p, Yc_p = centroid(img)

    #Loop the video clip and save the position of every key frame
    while vidcap.isOpened():
      previousImage = img
      success,img = vidcap.read()
      videoFrames.append(img)
      frameCounter += 1

      if success == False:
        keyFramesFrameCounter.append(frameCounter - 1)
        break

      if framesToSkip == 0:
        #X centroid and Y centroid
        Xc, Yc = centroid(img)

        if Xc != -1 and Yc != -1:
          #Distance
          L = math.sqrt((Xc_p - Xc)**2 + (Yc_p - Yc)**2)
          #Relative distance
          D = L * math.sqrt(Xc**2 + Yc**2)
          if ((abs(Xc_p - Xc) / Xc) <= e) and ((abs(Yc_p - Yc) / Yc) <= e):
            similarCount += 1
            sequence.append(img)
          else:
            if similarCount > m:
              #Calculate the position of the keyframe relative to the position of the current frame,
              #then skip enough frames to create space between clips
              keyFrameRelativePosition = findKeyFrame(sequence)
              keyFramesFrameCounter.append(frameCounter - keyFrameRelativePosition)
              framesToSkip = clip_size - keyFrameRelativePosition
            similarCount = 0
            sequence = []
          Xc_p = Xc
          Yc_p = Yc
        else:
          #TODO: Rimuovere questo print dopo aver verificato che funzioni
          print("Skipping a frame to create space...")
          framesToSkip -= 1

    print("Found ", len(keyFramesFrameCounter), " key frames, initiating clip extraction ------------------------------------------------------------------------------------------")

    i = 0
    #Loop through all the keyframes found and create a clip of clip_size length
    while i < len(keyFramesFrameCounter):
      keyFrames = createClip(keyFramesFrameCounter[i]-1, videoFrames, keyFrames, clip_size)
      i += 1

    #Write all the keyframes found in the directory specified
    for frame in keyFrames:
      if frame is not None:
        print("Writing frame number ", cont+1)
        cv2.imwrite(os.path.join(out_path,f"{cont}.jpg"), frame)
      cont +=1
    vidcap.release()
    cv2.destroyAllWindows()